In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
import pymongo

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'http://www.surfline.com/surf-reports-forecasts-cams/costa-rica/3624060'
browser.visit(url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

sidebar = soup.find('div', class_='sl-spot-list__spots')

locations = sidebar.find_all('h3', class_='sl-spot-details__name')
waveheights = sidebar.find_all('span', class_='quiver-surf-height')
loc_urls = sidebar.find_all('a', class_='sl-cam-list-link')


locations_list = []
waveheight_list = []
url_list = []
location_url_list = []



for location in locations:
    name = location.text.strip()
    locations_list.append(name)
    
    
for waveheight in waveheights:
    waveheight = waveheight.text.strip()
    waveheight_list.append(waveheight)
 

for loc_url in loc_urls:
    link = loc_url
    href = link['href']
    url_list.append(href)
              
location_url_list = ['http://www.surfline.com' + url for url in url_list]

location_url_list

['http://www.surfline.com/surf-report/camaronal/584204204e65fad6a7709140',
 'http://www.surfline.com/surf-report/puerto-sandino/5842041f4e65fad6a7708f1c',
 'http://www.surfline.com/surf-report/shack-s-shifty-s/584204204e65fad6a7709983',
 'http://www.surfline.com/surf-report/el-transito/584204204e65fad6a7709078',
 'http://www.surfline.com/surf-report/popoyo-area-rovercam/5842041f4e65fad6a7708db9',
 'http://www.surfline.com/surf-report/playa-maderas/5842041f4e65fad6a7708dc0',
 'http://www.surfline.com/surf-report/tamarindo/5842041f4e65fad6a7708b9b',
 'http://www.surfline.com/surf-report/samara/584204204e65fad6a770913d',
 'http://www.surfline.com/surf-report/las-pe-itas/5a1dfd8b1d8788001b7966de',
 'http://www.surfline.com/surf-report/miramar/584204204e65fad6a7709147',
 'http://www.surfline.com/surf-report/el-astillero/584204204e65fad6a770910c',
 'http://www.surfline.com/surf-report/popoyo-outer-reef/5842041f4e65fad6a7708dba',
 'http://www.surfline.com/surf-report/santana/5842041f4e65fad6a

In [5]:
water_temp_list = []
air_temp_list = []


for location_url in location_url_list:
    browser.visit(location_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all('div', class_='sl-wetsuit-recommender__weather')
    for res in results:
        try:
            water_temp = res.find_next('div').text
            water_temp_list.append(water_temp)
            air_temp = res.find_next('div').find_next("div").text
            air_temp_list.append(air_temp)
        except Exception as e:
            print(e)

water_temp_list

 

['84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '75 - 77 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF',
 '84 - 86 ºF']

In [6]:
surf_summary = pd.DataFrame(
    {'Location': locations_list,
     'WaveHeight': waveheight_list,
     'WaterTemp': water_temp_list,
     'AirTemp': air_temp_list,
     'URL': location_url_list
    })


surf_summary = surf_summary[['Location', 'WaveHeight', 'WaterTemp', 'AirTemp', 'URL']]

surf_summary.head()

,Location,WaveHeight,WaterTemp,AirTemp,URL
0,Camaronal,10-15FT,84 - 86 ºF,78 ºF,http://www.surfline.com/surf-report/camaronal/...
1,Puerto Sandino,8-12FT,84 - 86 ºF,81 ºF,http://www.surfline.com/surf-report/puerto-san...
2,Shack's/Shifty's,8-12FT,84 - 86 ºF,81 ºF,http://www.surfline.com/surf-report/shack-s-sh...
3,El Transito,6-10FT,84 - 86 ºF,81 ºF,http://www.surfline.com/surf-report/el-transit...
4,Popoyo Area Rovercam,6-10FT,84 - 86 ºF,80 ºF,http://www.surfline.com/surf-report/popoyo-are...


In [7]:
surf_dicts = surf_summary.to_dict('records')
print(surf_dicts)

[{'Location': 'Camaronal', 'WaveHeight': '10-15FT', 'WaterTemp': '84 - 86 ºF', 'AirTemp': '78 ºF', 'URL': 'http://www.surfline.com/surf-report/camaronal/584204204e65fad6a7709140'}, {'Location': 'Puerto Sandino', 'WaveHeight': '8-12FT', 'WaterTemp': '84 - 86 ºF', 'AirTemp': '81 ºF', 'URL': 'http://www.surfline.com/surf-report/puerto-sandino/5842041f4e65fad6a7708f1c'}, {'Location': "Shack's/Shifty's", 'WaveHeight': '8-12FT', 'WaterTemp': '84 - 86 ºF', 'AirTemp': '81 ºF', 'URL': 'http://www.surfline.com/surf-report/shack-s-shifty-s/584204204e65fad6a7709983'}, {'Location': 'El Transito', 'WaveHeight': '6-10FT', 'WaterTemp': '84 - 86 ºF', 'AirTemp': '81 ºF', 'URL': 'http://www.surfline.com/surf-report/el-transito/584204204e65fad6a7709078'}, {'Location': 'Popoyo Area Rovercam', 'WaveHeight': '6-10FT', 'WaterTemp': '84 - 86 ºF', 'AirTemp': '80 ºF', 'URL': 'http://www.surfline.com/surf-report/popoyo-area-rovercam/5842041f4e65fad6a7708db9'}, {'Location': 'Playa Maderas', 'WaveHeight': '6-10FT',

In [8]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [9]:
# Define database and collection
db = client.surfDB
collection = db.surf_summary
for surf_dict in surf_dicts:
    collection.insert_one(surf_dict)